##### Copyright 2018 The TensorFlow Authors.

# Generate Reviews with Cloud TPUs and Keras

This example uses [tf.keras](https://www.tensorflow.org/guide/keras) to build a *language model* and train it on a [Google Cloud TPU](https://cloud.google.com/tpu/). This language model predicts the next character of text given the text so far. The trained model can generate new snippets of text that read in a similar style to the text training data.

Note: To enable TPUs on Google Colab, select *Runtime > Change runtime type*, and set *Hardware acceleration* to TPU.

## Download data

In [0]:
f=open("AmazonChar.txt", 'r' , encoding='utf8')
contents1=f.read()
REVIEW_TXT=contents1
f.close()

In [0]:
f=open("ClothesChar.txt", 'r', encoding='utf8')
contents3=f.read()
REVIEW_TXT = contents3
f.close()

In [0]:
f=open("WineChar.txt", 'r', encoding='utf8')
contents2=f.read()
REVIEW_TXT= contents2
f.close()

### Build the data generator

In [5]:
import numpy as np
import six
import tensorflow as tf
import time
import os

# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

#SHAKESPEARE_TXT = '/content/shakespeare.txt'
#commented out because instead i'm using REVIEW_TXT

tf.logging.set_verbosity(tf.logging.INFO)

def transform(txt, pad_to=None):
  # drop any non-ascii characters
  output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  if pad_to is not None:
    output = output[:pad_to]
    output = np.concatenate([
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output

def training_generator(seq_len=100, batch_size=1024):
  """A generator yields (source, target) arrays for training."""
  #with tf.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    #txt = f.read()
  txt=REVIEW_TXT
    

  tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
  source = transform(txt)
  while True:
    offsets = np.random.randint(0, len(source) - seq_len, batch_size)

    # Our model uses sparse crossentropy loss, but Keras requires labels
    # to have the same rank as the input logits.  We add an empty final
    # dimension to account for this.
    yield (
        np.stack([source[idx:idx + seq_len] for idx in offsets]),
        np.expand_dims(
            np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
            -1),
    )

six.next(training_generator(seq_len=10, batch_size=1))

INFO:tensorflow:Input text [31550891] Aromas include tropical fruit, broom, brimstone an


(array([[114, 101,  97, 100, 121,  32, 116, 104, 101,  32]], dtype=int32),
 array([[[101],
         [ 97],
         [100],
         [121],
         [ 32],
         [116],
         [104],
         [101],
         [ 32],
         [ 99]]], dtype=int32))

## Build the model

The model is defined as a two-layer, forward-LSTM—with two changes from the `tf.keras` standard LSTM definition:

1. Define the input `shape` of our model which satisfies the [XLA compiler](https://www.tensorflow.org/performance/xla/)'s static shape requirement.
2. Use `tf.train.Optimizer` instead of a standard Keras optimizer (Keras optimizer support is still experimental).

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
  return model

## Train the model

The `tf.contrib.tpu.keras_to_tpu_model` function converts a `tf.keras` model to an equivalent TPU version. We then use the standard Keras methods to train: `fit`, `predict`, and `evaluate`.

In [7]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=10,
)
tpu_model.save_weights('/tmp/bard.h5', overwrite=True)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.7.20.242:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 5895018735270957403)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4427334198198249949)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 10731750899645578419)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9618771642461069739)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 468142364918972154)
INFO:tensorflow:*** Available Device: _DeviceAtt

## Make predictions with the model

Use the trained model to make predictions and generate your own Shakespeare-esque play.
Start the model off with a *seed* sentence, then generate 250 characters from it. We'll make five predictions from the initial seed.

In [8]:
BATCH_SIZE = 5
PREDICT_LEN = 300

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'the bottle of wine '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 is quite tannic, has enough personality, is short, grabby notes of cassis scents are followed by bright acidity and a bite of toast, and wood aromas aromas. Starts out sheer now, the wine is steely with intense acids cuts through the tone and oak has given the fruit flavors that streak to the finis

PREDICTION 1


 features arapal high fruit and balancing cherry and ckdery tannins. It tastes of thicknimento that contributes to the medium body on the palate, with gorgeous tones lie ending in a spicy richness and freshness. It is ready to drink. This wine starts off with redcherry texture and lively acidity car

PREDICTION 2


 of a coored golden surround balanncorice, yet textured and succulent wine, this broad wine brings a hint of more, brimming with big aromas of vanilla and dried fruit. Lighter than a halmockenh vintage, this sparkling wine has sharpherbill fruit, acidity ten or and lemon in the background. The palat

PREDICTION 3


 is densely tannic, with aromas of

In [0]:
import numpy as np
def transform(txt, pad_to=None):
  # drop any non-ascii characters
  output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  if pad_to is not None:
    output = output[:pad_to]
    output = np.concatenate([
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output

seed_txt = 'Wine '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), 5, axis=0)
seed

array([[ 87, 105, 110, 101,  32],
       [ 87, 105, 110, 101,  32],
       [ 87, 105, 110, 101,  32],
       [ 87, 105, 110, 101,  32],
       [ 87, 105, 110, 101,  32]], dtype=int32)